In [1]:
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import re
from datetime import date

In [140]:
#converts pdf, returns its text content as a string
def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text 

def preprocess(text):
    result = []
    
    # split to list of words
    result = text.split()
    
    return result

def checkLeapYear(year):  
    return (year%400==0)or(year%100!=0 and year%4==0)

def checkDate(day, month, year, minyear=1900, maxyear=2100):
    if not(minyear<=year<=maxyear):
        return False
    if month in [1, 3, 5, 7, 8, 10, 12]:
        if not(1<=day<=31):
            return False
    elif month in [4, 6, 9, 11]:
        if not(1<=day<=30):
            return False
    elif month in [2]:
        if not(1<=day<=28+checkLeapYear(year)):
            return False
    else:
        return False
    
    return True
    
def getDateFromString(datestring):
    result = datestring.replace('/', ' ').split()
    return int(result[0]), int(result[1]), int(result[2])
    
def checkDateString(datestring):
    day, month, year = getDateFromString(datestring)
    return checkDate(day, month, year)
    
def getCreateDatePart(text):
    result = ''
    begin = 0
    end = len(text)
    
    beginreg = ['ngày']
    endreg = ['năm']
    date_num_regex = '(19|20)\d{2}|([12]\d|3[01]|0?[1-9])'
    
    for reg in beginreg:
        found = re.search(reg, text, re.IGNORECASE)
        if found:
#             print(reg)
            begin = max(begin, found.start())
            
    for reg in endreg:
        found = re.search(reg, text[begin:], re.IGNORECASE)
        if found:
#             print(reg)
            end = min(end, begin + found.end())
            
#     print(begin, ', ', end)
    endfound = re.search(date_num_regex, text[end:], re.IGNORECASE)
    if endfound:
        end = end + endfound.end()
        
    result = text[begin:end]
    print(result)
#     print(begin, ', ', end)
    return result

def getSignedDatePart(text):
    result = ''
    begin = 0
    cost_regex = '\d{1,3}([.]\d{3})+'
    while True:
        found = re.search(cost_regex, text[begin:], re.IGNORECASE)
        if found:
            begin = begin + found.end()
        else:
            break
    result = text[begin:]
#     print(result)
    return result

def getCreateDate(text):
    result = ''
    dayregex = '([12]\d|3[01]|0?[1-9])'
    monthregex = '(1[012]|0?[1-9])'
    yearregex = '(19|20)\d{2}'
    daylist = []
    yearlist = []
    
    begin = 0
    end = len(text)

    while begin<end:
        found = re.search(yearregex, text[begin:], re.IGNORECASE)
        if found:
            yearlist.append(text[begin+found.start(): begin+found.end()])
#             print(text[begin+found.start(): begin+found.end()])
            text = text.replace(text[begin+found.start(): begin+found.end()], '    ', 1)
#             print(text)
            begin = begin + found.end()
        else:
            break
    begin = 0       
    while begin<end:
        found = re.search(dayregex, text[begin:], re.IGNORECASE)
        if found:
            daylist.append(text[begin+found.start(): begin+found.end()])
            begin = begin + found.end()
        else:
            break
    
    
    print(daylist)
    print(yearlist)
#     if len(yearlist)>0:
#         yearstring = yearlist[0]
#         currentyear = date.today().year    
#         for year in yearlist:
#             if (int(year)<=currentyear and currentyear-int(year)<currentyear-int(yearstring)):
#                 yearstring = year
#     else:
#         return result
    
    if yearlist:
        yearstring = yearlist[0]
        if len(daylist)>=2:
            n = len(daylist)
            for i in range(n-1):
                for j in range(i+1, n):
                    if re.match(monthregex, daylist[j]):
                        datestring = daylist[i] + '/' + daylist[j] + '/'+ yearstring
                        if checkDateString(datestring):
                            result = datestring
                            return result
        elif len(daylist)>0:
            if re.match(monthregex, daylist[0]):
                datestring = '1' + '/' + daylist[0] + '/'+ yearstring
                if checkDateString(datestring):
                    result = datestring
                    return result
    return result

def getSignedDate(text):
    result = ''
    
    regex = '\d{2}\/\d{2}\/\d{4}'
    begin = 0
    while True:
        found = re.search(regex, text[begin:], re.IGNORECASE)
        if found:
#             print(found)
            datestring = text[begin + found.start(): begin + found.end()]
            if checkDateString(datestring):
                result = datestring
            begin = begin + found.end()
        else:
            break
            
    return result

def getAllDate(filename):
    text = convert(filename)
    result = {}
    createDate = ''
    signedDate = ''
    
    createDatePart = getCreateDatePart(text)
    signedDatePart = getSignedDatePart(text)
    
    createDate = getCreateDate(createDatePart)
    signedDate = getSignedDate(signedDatePart)
    
    result.update({'createDate': createDate})
    result.update({'signedDate': signedDate})
    return result

In [146]:
filename = 'Data/39.pdf'
print(getAllDate(filename))

Ngày … tháng 10 năm 2019
['10']
['2019']
{'createDate': '1/10/2019', 'signedDate': ''}


In [91]:
text = convert(filename)

In [92]:
print(text)

 

HÓA ĐƠN GIÁ TRỊ GIA TĂNG 

(VAT INVOICE)pChuyenDoi 

Ngày (day) 13  tháng (month) 07 năm (year )2017 

 

Mẫu số (Form No.):   pMauSo 
Ký hiệu (Serial No.):   pKyHieu 
Số (Invoice No.):  

pSoHoaDon 

Đơn vị bán (Seller):   Tập ĐOÀN VIỄN THÔNG QUÂN ĐỘI 

MST (Tax Code):  

0100100900-045 

Địa chỉ (Address):  

 

Điện thoại (Tel.):  

                                       

STK (Account No.):  

 

Tại:  

Người mua (Buyer):   Phạm Thị BÍCH NGỌC 

Đơn vị (Co. name):     

MST (Tax Code):  

4200773861-2323 

Địa chỉ (Address):  

 

HTTT (Pay. method):             

STK (Account No.):  

 

Tại:  

 

STT 
(No.) 

Tên hàng hoá, dịch vụ 

(Description) 

ĐVT 
(Unit) 

Số lượng 
(Quantity) 

Đơn giá 
(Unit Price) 

Thuế suất 
(Tax rate) 

Tiền thuế 

(Tax) 

Thành tiền 

(Amount) 

1 

1 

2 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

2 

 

 

  

  

 

 

 

 

 

 

 

  

  

 

 

 

 

 

 

 

3 

  

  

 

 

 

 

 

 

 

4 

5 

6 = 4 x 5 

 

 

 

 

 

 

 

 


In [ ]:
def getCreateDate(text):
    
    result = ''
    vector = preprocess(text)
    dayregex = '^(0?[1-9]|[12]\d|3[01])$'
    monthregex = '(0?[1-9]|1[012])$'
    yearregex = '^(19|20)\d{2}$'
    
    daylist = list(filter(re.compile(dayregex).match, vector))
    yearlist = list(filter(re.compile(yearregex).match, vector))
    
    print(daylist)
    print(yearlist)
    if len(yearlist)>0:
        yearstring = yearlist[0]
        currentyear = date.today().year    
        for year in yearlist:
            if (int(year)<=currentyear and currentyear-int(year)<currentyear-int(yearstring)):
                yearstring = year
    else:
        return result
    
    if len(daylist)>=2 and len(yearlist):
        n = len(daylist)
        for i in range(n-1):
            for j in range(i+1, n):
                if re.match(monthregex, daylist[j]):
                    datestring = daylist[i] + '/' + daylist[j] + '/'+ yearstring
                    if checkDateString(datestring):
                        result = datestring
                        return result
                    
    return result